In [49]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
from matplotlib import pyplot as plt
from sklearn import model_selection

import codecs
import json
import os
SEED = 221

In [50]:
df = pd.read_csv("new.csv", sep="\t")
df = df.set_index("request_id", verify_integrity=True)
df = df.drop(['giver_user_flair'], axis=1)

df.head()

,giver_subreddits_at_request,giver_username,number_of_downvotes_of_request_at_retrieval,number_of_upvotes_of_request_at_retrieval,post_was_edited,request_number_of_comments_at_retrieval,request_text,request_title,request_url,requester_account_age_in_days_at_request,requester_has_verified_email,requester_num_pizza_given_at_request,requester_num_pizza_received_at_request,requester_num_pizza_related_comments_at_request,requester_num_pizza_related_posts_at_request,requester_received_pizza,requester_subreddits_at_request,requester_user_flair,requester_username,unix_timestamp_of_request
request_id,,,,,,,,,,,,,,,,,,,,
dphb2r,"['LetsNotMeet', 'Random_Acts_Of_Pizza']",bobbyshark,0,7,True,8,"hello! i've had a rough week, work is stressfu...",[REQUEST] really stressful week,https://www.reddit.com/r/Random_Acts_Of_Pizza/...,1939.0,True,1.0,1.0,15.0,3.0,1,"['IsTodayFridayThe13th', 'Wellthatsucks', 'Sub...",Trader (Got 2 Gave 1) Fish: 150,GojiraWho,1.572489e+09
dpccso,"['holdmyfries', 'PetTheDamnKitty', 'SuicideWat...",SanaeKojima,0,4,False,14,i posted on r/borrow as im trying to pay my li...,[Request] could use a nice pie if possible,https://www.reddit.com/r/Random_Acts_Of_Pizza/...,495.0,True,0.0,0.0,0.0,1.0,1,"['DirtySnapchat', 'funny', 'Idubbbz', 'mildlyi...",Recipient (Got 1) Fish: 115,pookumz123,1.572467e+09
dofjc3,"['funny', 'politics', 'photography', '2XLookbo...",shpooey,0,57,False,24,"with rent coming up on friday, i’ve already go...","[Request] Vancouver, Canada. Low-income youth ...",https://www.reddit.com/r/Random_Acts_Of_Pizza/...,391.0,True,0.0,0.0,1.0,5.0,1,"['BlackDesertMobile', 'freebies', 'GetEmployed...",Recipient (Got 1) Fish: 175,Pd0xG,1.572297e+09
dofige,"['holdmyfries', 'PetTheDamnKitty', 'SuicideWat...",sanaekojima,0,4,False,8,hi i have tried to refrain from making a reque...,[Request] 37343 broke no car and no food in my...,https://www.reddit.com/r/Random_Acts_Of_Pizza/...,1196.0,True,0.0,1.0,7.0,6.0,1,"['UpliftingNews', 'FortNiteBR', 'Jokes', 'funn...",Recipient (Got 2) Fish: 140,tvacnaar,1.572297e+09
doeex3,"['LetsNotMeet', 'Random_Acts_Of_Pizza']",bobbyshark,0,16,False,20,been struggling this month im living from payc...,[request] im struggling right now even pizza i...,https://www.reddit.com/r/Random_Acts_Of_Pizza/...,307.0,True,0.0,0.0,0.0,1.0,1,"['confusing_perspective', 'PokemonSwordAndShie...",Recipient (Got 1),bxsmack09,1.572293e+09


In [54]:

df = df[df['request_text'].notnull()]

In [56]:
df.shape

(4524, 20)

In [57]:
print("success_rate: ", 100.0 * sum(df['requester_received_pizza']) / float(df.shape[0]))

success_rate:  57.97966401414677


# Make splits while maintaining class balance

In [58]:
splits = [0.6, 0.2, 0.2]

In [59]:
train_and_val, test_set = sklearn.model_selection.train_test_split(
    df, test_size=0.2, stratify=df.requester_received_pizza, random_state=SEED)

In [60]:
train_set, val_set = sklearn.model_selection.train_test_split(
    train_and_val, test_size=0.25, stratify=train_and_val.requester_received_pizza,
    random_state=SEED)

In [61]:
train_mini, _ = sklearn.model_selection.train_test_split(
    train_set, train_size=0.01, stratify=train_set.requester_received_pizza, random_state=SEED)
val_mini, _ = sklearn.model_selection.train_test_split(
    val_set, train_size=0.01, stratify=val_set.requester_received_pizza, random_state=SEED)
test_mini, _ = sklearn.model_selection.train_test_split(
    test_set, train_size=0.01, stratify=test_set.requester_received_pizza, random_state=SEED)

train_medium, _ = sklearn.model_selection.train_test_split(
    train_set, train_size=0.1, stratify=train_set.requester_received_pizza, random_state=SEED)
val_medium, _ = sklearn.model_selection.train_test_split(
    val_set, train_size=0.1, stratify=val_set.requester_received_pizza, random_state=SEED)
test_medium, _ = sklearn.model_selection.train_test_split(
    test_set, train_size=0.1, stratify=test_set.requester_received_pizza, random_state=SEED)

In [62]:
len(train_set) + len(val_set) + len(test_set), len(df)

(4524, 4524)

In [63]:
len(train_mini) + len(val_mini) + len(test_mini), len(df)

(45, 4524)

In [64]:
sum(train_set['requester_received_pizza'])/len(train_set)

0.5795873249815771

In [65]:
sum(test_set['requester_received_pizza'])/len(test_set)

0.580110497237569

In [66]:
sum(val_set['requester_received_pizza'])/len(val_set)

0.580110497237569

In [67]:
train_set.to_csv( "train_all.csv", index_label="request_id")
val_set.to_csv( "val_all.csv", index_label="request_id")
test_set.to_csv("test_all.csv", index_label="request_id")
    
train_medium.to_csv("train_medium.csv", index_label="request_id")
val_medium.to_csv("val_medium.csv", index_label="request_id")
test_medium.to_csv( "test_medium.csv", index_label="request_id")
    
train_mini.to_csv( "train_mini.csv", index_label="request_id")
val_mini.to_csv("val_mini.csv", index_label="request_id")
test_mini.to_csv("test_mini.csv", index_label="request_id")